In [1]:
import importlib
from tqdm import tqdm
from tqdm import trange

import pentago
from pentago import agent, memory, minimax, learning_agent, neural_model, monte_carlo, monte_carlo_multi, evaluate, game

In [ ]:
initial_view = memory.MemoryView()
controller = pentago.Controller([minimax.MinimaxSearchAgent(depth=2)] * 2, initial_view)
for i in tqdm(range(1000)):
    controller.play_game()

 43%|████▎     | 431/1000 [19:12<25:17,  2.67s/it]

In [ ]:
import dill as pickle

In [ ]:
pickle.dump(initial_view, open('initial_view.pkl', mode='wb'))

In [ ]:
initial_view = pickle.load(open('initial_view.pkl', mode='rb'))

In [ ]:
importlib.reload(neural_model)
importlib.reload(learning_agent)
importlib.reload(game)
importlib.reload(monte_carlo_multi)
neural_agent = learning_agent.NeuralAgent(model_params={'dense_regularization_const': 1e-2,
                                                        'regularization_const': 0,
                                                        'num_layers': 5,
                                                        'kernel_size': 3,
                                                        'num_filters': 64})

In [ ]:
print(evaluate.tally_wins([agent.RandomAgent(), neural_agent], trials=10))
print(evaluate.tally_wins([minimax.MinimaxSearchAgent(), neural_agent], trials=10, verbose=True))

In [ ]:
neural_agent.fit(initial_view.get_experiences())

In [ ]:
print(evaluate.tally_wins([agent.RandomAgent(), neural_agent], trials=10))
print(evaluate.tally_wins([minimax.MinimaxSearchAgent(), neural_agent], trials=10))

In [ ]:
while True:
    view = memory.MemoryView()
    controller = pentago.Controller([neural_agent] * 2, view)
    with trange(500) as t:
        for i in t:
            controller.play_game()
    neural_agent.fit(view.get_experiences())
    wins = evaluate.tally_wins([minimax.MinimaxSearchAgent(), neural_agent], trials=10)
    print(wins)
    if wins[-1] > 7:
        break